<a href="https://colab.research.google.com/github/HPavlova/data_science-hw-06/blob/main/hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd

from pyspark.context import SparkContext
from pyspark.sql import Row
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("k1", "v1").getOrCreate()

df = spark.read.csv('data/bestsellers with categories.csv')
pd_df = df.toPandas()

AnalysisException: ignored